In [1]:
require(stringr)
project_name = 'ML-MADNESS'
repo_path = file.path(substr(getwd(),0,gregexpr(pattern = project_name ,
                                                getwd())[[1]][1]-2),
                      project_name)
setwd(repo_path)

Loading required package: stringr



In [24]:
source(file.path(repo_path,'objects','helper_functions.R'))
p = scan(file.path(repo_path,'features/requirements.txt'),what="",sep="\n")
load_all_packages(p)
p = scan(file.path(repo_path,'sagemaker/requirements.txt'),what="",sep="\n")
load_all_packages(p)
slice = dplyr::slice
rename = dplyr::rename


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [3]:
# Sagemaker Settings
sagemaker <- import('sagemaker')
session <- sagemaker$Session()
bucket <- session$default_bucket()
role_arn <- sagemaker$get_execution_role()

In [4]:
source(file.path(repo_path,'features','api_calls.r'))

In [17]:
source(file.path(repo_path,'features','modeling.r'))
boxscores = load_box_score_data()

In [30]:
all_team_data = initial_cleaning(boxscores)

In [33]:
team_info = all_team_data %>%
  ungroup() %>% 
  select(team_id,team_display_name.x) %>% 
  group_by(team_id,team_display_name.x) %>% 
  summarize(APPEARANCES = n()) %>% 
  rename("team_name" = "team_display_name.x") %>% 
  arrange(desc(APPEARANCES))

team_data_refined = all_team_data %>% 
  ungroup() %>% 
  select(-c(team_display_name.x,team_display_name.y))

`summarise()` has grouped output by 'team_id'. You can override using the
`.groups` argument.


In [48]:
source(file.path(repo_path,'features','elo_helper.r'))

In [53]:
df = BOOSTED_calculate_relative_elo(team_data_refined = team_data_refined %>% 
                                filter(SEASON==2022),
                            features = c("PTS.x","PTS.y"),
                            rename_fs = c("PTS_SCORED","PTS_ALLOWED"))

In [54]:
tail(df)

game_date,game_id,team_id,opponent_id,APPEARANCE,OPP_APPEARANCE,PTS.x,PTS.y,PTS_SCORED_AVG,PTS_SCORED_SD,⋯,PTS_SCORED_DEF_ELO,PTS_SCORED_ELO_NEXT,PTS_SCORED_DEF_ELO_NEXT,PTS_ALLOWED_AVG,PTS_ALLOWED_SD,PTS_ALLOWED_PERF,PTS_ALLOWED_ELO,PTS_ALLOWED_DEF_ELO,PTS_ALLOWED_ELO_NEXT,PTS_ALLOWED_DEF_ELO_NEXT
<date>,<int>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2022-04-02,401408634,222,2305,38,39,65,81,67.85714,67.85714,⋯,2303.809,1434.775,2269.215,65.57143,65.57143,0.23529412,1400.181,2303.809,1434.775,2269.215
2022-04-02,401408634,2305,222,39,38,81,65,74.75000,74.75000,⋯,2097.920,1822.469,2105.573,68.50000,68.50000,-0.05109489,1830.122,2097.920,1822.469,2105.573
2022-04-03,401408635,153,150,38,39,81,77,67.85714,67.85714,⋯,2181.780,2089.144,2155.643,65.57143,65.57143,0.17429194,2063.007,2181.780,2089.144,2155.643
2022-04-03,401408635,150,153,39,38,77,81,74.75000,74.75000,⋯,1937.582,1833.482,1910.221,68.50000,68.50000,0.18248175,1806.121,1937.582,1833.482,1910.221
2022-04-05,401408636,153,2305,39,40,69,72,74.75000,74.75000,⋯,2105.573,2096.808,2097.909,68.50000,68.50000,0.05109489,2089.144,2105.573,2096.808,2097.909
2022-04-05,401408636,2305,153,40,39,72,69,72.00000,72.00000,⋯,2155.643,1818.254,2159.858,71.00000,71.00000,-0.02816901,1822.469,2155.643,1818.254,2159.858


In [57]:
predictors = c('PTS_SCORED_ELO','PTS_SCORED_DEF_ELO','PTS_ALLOWED_ELO','PTS_ALLOWED_DEF_ELO')

modeling = df %>% 
            filter(APPEARANCE!=1,
            OPP_APPEARANCE!=1) %>%
            select(PTS.x,PTS.y,game_date,game_id,team_id,opponent_id,predictors)%>%
            mutate(MARGIN = PTS.x - PTS.y,
                   WIN = ifelse(MARGIN >0,1,0))
tail(modeling)

PTS.x,PTS.y,game_date,game_id,team_id,opponent_id,PTS_SCORED_ELO,PTS_SCORED_DEF_ELO,PTS_ALLOWED_ELO,PTS_ALLOWED_DEF_ELO,MARGIN,WIN
<dbl>,<dbl>,<date>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
65,81,2022-04-02,401408634,222,2305,1400.181,2303.809,1400.181,2303.809,-16,0
81,65,2022-04-02,401408634,2305,222,1830.122,2097.920,1830.122,2097.920,16,1
81,77,2022-04-03,401408635,153,150,2063.007,2181.780,2063.007,2181.780,4,1
77,81,2022-04-03,401408635,150,153,1806.121,1937.582,1806.121,1937.582,-4,0
69,72,2022-04-05,401408636,153,2305,2089.144,2105.573,2089.144,2105.573,-3,0
72,69,2022-04-05,401408636,2305,153,1822.469,2155.643,1822.469,2155.643,3,1


[1] "game_date"               "game_id"                
 [3] "team_id"                 "opponent_id"            
 [5] "APPEARANCE"              "OPP_APPEARANCE"         
 [7] "PTS.x"                   "F_AVG"                  
 [9] "F_SD"                    "PERF"                   
[11] "PTS_SCORED_ELO"          "PTS_SCORED_DEF_ELO"     
[13] "PTS_SCORED_ELO_NEXT"     "PTS_SCORED_DEF_ELO_NEXT"